In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import seaborn as sns
import re
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

### Data Collection

In [ ]:
import requests

# URLs of the files
train_data_url = 'https://www.raphaelcousin.com/modules/module5/exercise/module5_exercise_train.csv'
test_data_url = 'https://www.raphaelcousin.com/modules/module5/exercise/module5_exercise_test.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_data_url, 'module5_exercise_train.csv')
download_file(test_data_url, 'module5_exercise_test.csv')

In [ ]:
df_train =  pd.read_csv("module5_exercise_train.csv", sep=",")
df_test =  pd.read_csv("module5_exercise_test.csv", sep=",")

### Data analysis

In [ ]:
#### Make a complete analysis on data preprocessing
# Inconsistencies 数据类型或值不一致
# Duplicates (data.duplicated().sum()) 重复
# Missing values (data.isnull().sum()) 缺失值
# Categorical 分类变量及其分布
# Outliers 数值特征中的异常值
# Feature Engineering 特征工程的潜力
# Feature Selection and/or Dimensionality Reduction 特征选择或降维的机会

In [ ]:
#pd.concat将df_train和df_test按（行）纵向拼接
data = pd.concat([df_train, df_test], axis=0)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
data.shape

In [ ]:
#绘制指定特征在给定日期范围内的时间序列图
def plot_feature_over_time(df, feature, date_id_start, date_id_end):
    df_filtered = df[(df['date'] >= date_id_start) & (df['date'] <= date_id_end)]
    
    if feature not in df_filtered.columns:
        print(f"Feature '{feature}' not found in the DataFrame.")
        return
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df_filtered['date'], df_filtered[feature], label=feature, linestyle='-')
    plt.xlabel('Date')
    plt.ylabel(feature)
    plt.title(f'{feature} from {date_id_start} to {date_id_end}')
    plt.xticks(rotation=45) #横轴刻度旋转 45 度，避免文字重叠
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
data['date'] = pd.to_datetime(data['date'])
#将 data 数据框中的 date 列转换为 datetime 类型

#### Detecting Inconsistencies

In [ ]:
features = df_train.columns
print(features)

In [ ]:
data_date = pd.DataFrame(data["date"])
print(data_date.applymap(lambda x: not re.match(r"\d{4}-\d{2}-\d{2}", str(x))).sum().sum())

In [ ]:
data.info()

In [ ]:
kmh_count=data['wind_speed'].str.contains('km/h').sum()
ms_count=data['wind_speed'].str.contains('m/s').sum()

print(f"Number of 'km/h' in wind_speed: {kmh_count}")
print(f"Number of 'm/s' in wind_speed: {ms_count}")

We should standardize the units in 'wind_speed'.

In [ ]:
def convert_to_kmh(value:str):
    if not isinstance(value, str) or pd.isna(value):
        return None
    if 'km/h' in value:
        return float(value.split()[0])
    elif 'm/s' in value:
        return float(value.split()[0]) * 3.6
    else:
        return None

In [ ]:
data['wind_speed'] = data['wind_speed'].apply(convert_to_kmh)

#### Detecting duplicates

In [ ]:
exact_duplicates = data[data.duplicated()]
print(exact_duplicates)

In [ ]:
def handle_duplicates(df):
    df_no_duplicates = df.drop_duplicates()
    return df_no_duplicates

In [ ]:
data = handle_duplicates(data)

#### Detecting Missing Values

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title('Heatmap of Missing Values')
plt.show()

Except for the 'oil_brent_price_indicator' column, all other columns have missing values.

For the imputation of the categorical variable 'weather_condition', we decide to determine it based on its humidity.

After a preliminary observation, we need to first address the outliers in **humidity** and **electricity_demand**.

In [ ]:
outlier_indice_humidity=data[data['humidity'] > 200].index
for i in outlier_indice_humidity:
    print(data.loc[i])

In [ ]:
valid_mean_sunny = data[(data['weather_condition'] == 'Sunny') & (data['humidity'] <= 1000)]['humidity'].mean()
data.loc[(data['weather_condition'] == 'Sunny') & (data['humidity'] > 1000), 'humidity'] = valid_mean_sunny

valid_mean_rainy = data[(data['weather_condition'] == 'Rainy') & (data['humidity'] <= 1000)]['humidity'].mean()
data.loc[(data['weather_condition'] == 'Rainy') & (data['humidity'] > 1000), 'humidity'] = valid_mean_rainy

sns.boxplot(x=data['humidity'])
plt.title('Box Plot for Outlier Detection')
plt.show()

In [ ]:
sns.boxplot(x=data['electricity_demand'])
plt.title('Box Plot for Outlier Detection')
plt.show()

In [ ]:
data.loc[data['electricity_demand'] <= 0, 'electricity_demand'] = abs(data['electricity_demand'])/1000

In [ ]:
sns.boxplot(x=data['electricity_demand'])
plt.title('Box Plot for Outlier Detection')
plt.show()

Now we can start handling the missing values.

In [ ]:
def fill_weather_condition(weather_series):
    weather_series = weather_series.copy()  # 避免修改原数据
    n = len(weather_series)
    
    i = 0
    while i < n:
        if pd.isna(weather_series.iloc[i]):  # 如果当前值是缺失值
            start = i  # 记录缺失值的起始位置
            
            # 找到连续缺失值的结束位置
            while i < n and pd.isna(weather_series.iloc[i]):
                i += 1
            end = i  # 连续缺失值的结束位置（不包含）
            
            # 获取前后值
            prev_value = weather_series.iloc[start - 1] if start > 0 else None
            next_value = weather_series.iloc[end] if end < n else None
            
            # 填充逻辑
            if prev_value == next_value and pd.notna(prev_value):
                # 前后值相同，填充相同值
                weather_series.iloc[start:end] = prev_value
            elif pd.notna(prev_value) and pd.notna(next_value):
                # 前后值不同，随机选择一个值填充
                weather_series.iloc[start:end] = np.random.choice([prev_value, next_value])
            elif pd.notna(prev_value):
                # 只有前值存在，用前值填充
                weather_series.iloc[start:end] = prev_value
            elif pd.notna(next_value):
                # 只有后值存在，用后值填充
                weather_series.iloc[start:end] = next_value
        else:
            i += 1  # 非缺失值，继续向后遍历
    
    return weather_series


In [ ]:
data['weather_condition'] = fill_weather_condition(data['weather_condition'])

In [ ]:
# 按 weather_condition 分组，计算 humidity 的均值
weather_condition_mean = data.groupby('weather_condition')['humidity'].mean()

# 用对应的均值填补缺失值
data['humidity'] = data.apply(
    lambda row: weather_condition_mean[row['weather_condition']] if pd.isna(row['humidity']) else row['humidity'],
    axis=1
)

In [ ]:
# 按 weather_condition 分组，计算 wind_speed 的均值
weather_condition_mean_wind_speed = data.groupby('weather_condition')['wind_speed'].mean()

# 用对应的均值填补 wind_speed 的缺失值
data['wind_speed'] = data.apply(
    lambda row: weather_condition_mean_wind_speed[row['weather_condition']] if pd.isna(row['wind_speed']) else row['wind_speed'],
    axis=1
)

In [ ]:
# 提取 temperature_station1 到 temperature_station10 的列名
temperature_columns = [f'temperature_station{i}' for i in range(1, 11)]

# 绘制折线图
plt.figure(figsize=(14, 8))
for col in temperature_columns:
    plt.plot(data.index, data[col], label=col, alpha=0.7)  # 添加透明度以区分线条

# 设置图表标题和轴标签
plt.title('Temperature Trends Across Stations', fontsize=16)
plt.xlabel('Index', fontsize=14)
plt.ylabel('Temperature', fontsize=14)
plt.legend(title='Stations', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 将 date 列设置为索引
data = data.set_index('date')

# 使用时间插值填补缺失值
for col in [f'temperature_station{i}' for i in range(1, 11)]:
    data[col] = data[col].interpolate(method='time')

# 如果需要恢复索引
data = data.reset_index()

In [ ]:
# 获取除了 temperature_station2 的其他列名
stations = [f'temperature_station{i}' for i in range(1, 11) if i != 2]

# 计算其他站点的均值并填补 temperature_station2 前两个缺失值
data.loc[:1, 'temperature_station2'] = data.loc[:1, stations].mean(axis=1)

In [ ]:
plt.figure(figsize=(14, 8))
for col in temperature_columns:
    plt.plot(data.index[0:100], data[col][0:100], label=col, alpha=0.7)  # 添加透明度以区分线条

# 设置图表标题和轴标签
plt.title('Temperature Trends Across Stations', fontsize=16)
plt.xlabel('Index', fontsize=14)
plt.ylabel('Temperature', fontsize=14)
plt.legend(title='Stations', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
data.info()

#### Detecting Outliers

In [ ]:
print(data.dtypes)

In [ ]:
plt.figure(figsize=(10, 6))
for column in data.columns:
    if column != 'oil_brent_price_indicator':
        sns.boxplot(x=data[column])
        plt.title('Box Plot for Outlier Detection')
        plt.show()

#### Detecting Categorical Values

For the two categorical variables, `oil_brent_price_indicator` and `weather_condition`, we will convert them into numerical variables.

In [ ]:
data_encoded = pd.get_dummies(data, columns=['oil_brent_price_indicator', 'weather_condition'])
bool_columns = data_encoded.select_dtypes(include=['bool']).columns
data_encoded[bool_columns] = data_encoded[bool_columns].astype(float)

data_encoded.info()

#### Feature Engineering

##### DateTime Decomposition

In [ ]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour

# 检查处理结果
print(data[['date', 'year', 'month', 'day', 'hour']].head())


##### Domain-Specific Feature Extraction

In [ ]:
for i in range(1, 11):
    station_col = f'temperature_station{i}'
    feel_col = f'temperature_feel_station{i}'
    data[feel_col] = data[station_col] - 0.55 * (1 - data['humidity'] / 100) * (data[station_col] - 14.5)

# 检查结果
feel_columns = [f'temperature_feel_station{i}' for i in range(1, 11)]
print(data[feel_columns].head())

In [ ]:
data.info()

#### Feature Selection and Dimensionality Reduction

In [ ]:
print(data.columns)

In [ ]:
data_with_demand = data[data['electricity_demand'].notna()].copy()
data_without_demand = data[data['electricity_demand'].isna()].copy()

In [ ]:
X=data_with_demand.drop(columns=['electricity_demand','date','weather_condition','oil_brent_price_indicator'])
y=data_with_demand['electricity_demand']

selector = SelectKBest(f_classif, k=5)  # Select top 5 features
X_new = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_features = selector.get_support(indices=True)
feature_names = X.columns[selected_features]
print("Selected features:", feature_names)

In [ ]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

# 模型训练
model.fit(X_train, y_train)

# 模型预测
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 计算误差
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# 输出结果
print(f"Train MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")


### Data Preprocessing Evaluation Strategy

In [ ]:
# Provide a complete data preprocessing transformations

In [ ]:
# # 1. Handle Inconsistencies
# def handle_inconsistencies(X_train, y_train):
#     X_train_no_inconsistencies = X_train.copy()
#     X_train_no_inconsistencies['date'] =pd.to_datetime(X_train['date'])
#     X_train_no_inconsistencies['wind_speed'] = X_train_no_inconsistencies['wind_speed'].apply(convert_to_kmh)
#     return X_train_no_inconsistencies, y_train.copy()

# # 2. Handling Duplicates
# def handle_duplicates_2(X_train, y_train, X_val=None):
#     X_train_no_duplicates = X_train.drop_duplicates()
#     y_train_no_duplicates = y_train[X_train_no_duplicates.index]
#     return X_train_no_duplicates, y_train_no_duplicates

# # 3. Handling Outliers
# def handle_outliers(X_train, y_train, X_val=None):
#     if X_val is not None:
#         #humidity
#         valid_mean_sunny_def = X_train[(X_train['weather_condition'] == 'Sunny') & (X_train['humidity'] <= 1000)]['humidity'].mean()
#         X_train.loc[(X_train['weather_condition'] == 'Sunny') & (X_train['humidity'] > 1000), 'humidity'] = valid_mean_sunny_def

#         valid_mean_rainy_def = X_train[(X_train['weather_condition'] == 'Rainy') & (X_train['humidity'] <= 1000)]['humidity'].mean()
#         X_train.loc[(X_train['weather_condition'] == 'Rainy') & (X_train['humidity'] > 1000), 'humidity'] = valid_mean_rainy_def

#         #electricity_demand
#         X_train.loc[X_train['electricity_demand'] <= 0, 'electricity_demand'] = abs(X_train['electricity_demand'])/1000

#         return X_train.copy(), y_train, X_val.copy()
#     else:
#         return X_train.copy(), y_train

# # 4. Handling Missing Values
# def handle_missing_values(X_train, y_train, X_val=None):
#     if X_val is not None:
#         X_train['weather_condition'] = X_train['weather_condition'].apply(fill_weather_condition).fillna(-1)
#         X_val = X_val.fillna(-1)
#         return X_train.copy(), X_val.copy()
#     else:
#         X_train['weather_condition'] = X_train['weather_condition'].apply(fill_weather_condition).fillna(-1)
#         return X_train
    
# # 5. Handling Categorical Values
# def handle_categorical(X_train, y_train, X_val=None):
#     if X_val is not None:
#         return X_train.copy(), X_val.copy()
#     else:
#         return X_train.copy()

# # 6. Feature Engineering
# def feature_engineering(X_train, y_train, X_val=None):
#     if X_val is not None:
#         return X_train.copy(), y_train, X_val.copy()
#     else:
#         return X_train.copy(), y_train

# # 7. Feature Selection and Dimensionality Reduction
# def feature_selection(X_train, y_train, X_val=None):
#     selected_columns = ['humidity', 'temperature_station1',
#        'temperature_station2', 'temperature_station3', 'temperature_station4',
#        'temperature_station5', 'temperature_station6', 'temperature_station7',
#        'temperature_station8', 'temperature_station9', 'temperature_station10']
#     if X_val is not None:
#         return X_train[selected_columns], X_val[selected_columns]
#     else:
#         return X_train[selected_columns]

In [ ]:
# def evaluate_pipeline(X, y, n_splits=5):

#     ### call transformations here, if there is no learning and no need to be crossval
#     X, y = handle_inconsistencies(X, y)
#     # X, y = handle_duplicates(X, y)
#     X  = handle_missing_values(X, y)
#     # X_train = handle_categorical(X, y)
#     X, y = handle_outliers(X, y)
#     # X, y = feature_engineering(XX, y)
#     X = feature_selection(X, y)
    
#     model = LinearRegression()
    
#     tscv = TimeSeriesSplit(n_splits=n_splits)
    
#     train_scores = []
#     val_scores = []
    
#     for fold, (train_index, val_index) in enumerate(tscv.split(X)):
#         print(f"Processing fold {fold + 1}/{n_splits}...")
        
#         # Split data into train and validation sets
#         X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
#         y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

#         ### call transformations here, if there is learning
#         # X_train, y_train, X_val = handle_inconsistencies(X_train, y_train, X_val)
#         X_train, y_train, X_val = handle_duplicates(X_train, y_train, X_val)
#         # X_train, X_val = handle_missing_values(X_train, y_train, X_val)
#         X_train, X_val = handle_categorical(X_train, y_train, X_val)
#         # X_train, y_train, X_val = handle_outliers(X_train, y_train, X_val)
#         X_train, y_train, X_val = feature_engineering(X_train, y_train, X_val)
#         # X_train, X_val = feature_selection(X_train, y_train, X_val)
        
#         # Train the model
#         model.fit(X_train, y_train)
        
#         # Predict on training set
#         y_train_pred = model.predict(X_train)
#         train_mse = mean_squared_error(y_train, y_train_pred)
#         train_scores.append(train_mse)
        
#         # Predict on validation set
#         y_val_pred = model.predict(X_val)
#         val_mse = mean_squared_error(y_val, y_val_pred)
#         val_scores.append(val_mse)
        
#         print(f"Fold {fold + 1} Train MSE: {train_mse:.4f}, Validation MSE: {val_mse:.4f}")
    
#     # Compute mean, max, and min values for train and validation MSE
#     mean_train_mse = np.mean(train_scores)
#     max_train_mse = np.max(train_scores)
#     min_train_mse = np.min(train_scores)
    
#     mean_val_mse = np.mean(val_scores)
#     max_val_mse = np.max(val_scores)
#     min_val_mse = np.min(val_scores)
    
#     # Print results
#     print("\nTrain MSE:")
#     print(f"Mean: {mean_train_mse:.4f}, Max: {max_train_mse:.4f}, Min: {min_train_mse:.4f}")
    
#     print("\nValidation MSE:")
#     print(f"Mean: {mean_val_mse:.4f}, Max: {max_val_mse:.4f}, Min: {min_val_mse:.4f}")
    
#     return mean_val_mse  # Return mean validation MSE as the overall score

In [ ]:
# # Prepare X and y
# X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
# y = df_train.copy().pop('electricity_demand')

# # Run the evaluation
# evaluate_pipeline(X, y)

### Generating Submission File

In [ ]:
# Train and submit your results

In [ ]:
# # Prepare X_train and y_train from your data
# df_train =  pd.read_csv("module5_exercise_train.csv", sep=",")

# X_train = df_train.drop(columns=['electricity_demand'], axis=1)
# y_train = df_train['electricity_demand']

# X_test =  pd.read_csv("module5_exercise_test.csv", sep=",")

In [ ]:
# def train_and_predict_to_submit(X_train, y_train, X_test):
#     model = LinearRegression()
    
#     X_train, y_train, X_test = handle_inconsistencies(X_train, y_train, X_test)
#     X_train, y_train, X_test = handle_duplicates(X_train, y_train, X_test)
#     X_train, X_test = handle_missing_values(X_train, y_train, X_test)
#     X_train, X_test = handle_categorical(X_train, y_train, X_test)
#     X_train, y_train, X_test = handle_outliers(X_train, y_train, X_test)
#     X_train, y_train, X_test = feature_engineering(X_train, y_train, X_test)
#     X_train, X_test = feature_selection(X_train, y_train, X_test)

#     # Train the model on the entire training set
#     print(f"Training model on entire dataset of shape: {X_train.shape}")
#     model.fit(X_train, y_train)
    
#     # Predict on the test set
#     print(f"Predicting on test dataset of shape: {X_test.shape}")
#     y_test_pred = model.predict(X_test)
    
#     return y_test_pred

In [ ]:
# Call serve_model to train and predict
X_test_after_pocess=data_without_demand.drop(columns=['electricity_demand','date','weather_condition','oil_brent_price_indicator'])
y_test_pred = model.predict(X_test_after_pocess)

In [ ]:
# Generating Submission File
submission = pd.DataFrame({
    'date': data_without_demand['date'],
    'electricity_demand': y_test_pred
})

# Save the submission file
submission.to_csv('submission.csv', index=False, sep=',')
print("Submission file saved as 'submission.csv'.")